In [4]:
import math

In [19]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.

This is by no means production code.
"""
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
import requests

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(2, 3) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
# generate list of urls to visit
regions = ["northern-victoria-vic","melbourne-region-vic","eastern-suburbs-vic","south-western-victoria-vic","gippsland-greater-region-vic"]
for region in regions:
    url = BASE_URL + f"/rent/{region}/?sort=price-desc&page=1"
    bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")
    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    result_count = int(bs_object \
        .find(
            "h1",
            {"data-testid": "summary","class":"css-ekkwk0"}
        ).text.split()[0])
    if result_count < 1000:
        num_pages = range(1,math.ceil(result_count/20))
    else:
        num_pages = range(1,50)
    print(region, result_count,num_pages)
    for page in num_pages:
        url = BASE_URL + f"/rent/{region}/?sort=price-desc&page={page}"
        bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")

        # find the unordered list (ul) elements which are the results, then
        # find all href (a) tags that are from the base_url website.
        index_links = bs_object \
            .find(
                "ul",
                {"data-testid": "results"}
            ) \
            .findAll(
                "a",
                href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
            )

        for link in index_links:
            # if its a property address, add it to the list
            if 'address' in link['class']:
                url_links.append(link['href'])


northern-victoria-vic 831 range(1, 42)
melbourne-region-vic 11628 range(1, 50)
eastern-suburbs-vic 1642 range(1, 50)
south-western-victoria-vic 500 range(1, 25)
gippsland-greater-region-vic 537 range(1, 27)


In [20]:
len(url_links)

3780

In [30]:
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")
    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text
    property_metadata[property_url]['type'] = bs_object \
        .find("div", {"data-testid": "listing-summary-property-type"}) \
        .text
    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]

    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]


# output to example json in data/raw/
with open('../data/raw/domain_scrap_region.json', 'w') as f:
    dump(property_metadata, f)



    

In [16]:
import json

In [34]:
# Opening JSON file
f = open('../data/raw/domain_scrap_region.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [35]:
len(data)

3779